In [48]:
!pip install xlrd>=2.0.1
!pip install pyLDAvis

# Import Libraries 

In [50]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import gensim
from gensim import corpora
from gensim.models import LdaModel
import pyLDAvis
import pyLDAvis.gensim
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nomat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nomat\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nomat\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Load Data

In [52]:
data = pd.read_excel('savedrecs.xls', engine='xlrd') 
data.head()

,Publication Type,Authors,Book Authors,Book Editors,Book Group Authors,Author Full Names,Book Author Full Names,Group Authors,Article Title,Source Title,...,Web of Science Index,Research Areas,IDS Number,Pubmed Id,Open Access Designations,Highly Cited Status,Hot Paper Status,Date of Export,UT (Unique WOS ID),Web of Science Record
0,J,"Olaya, JFC; Corrales, JC",NaN,NaN,NaN,"Olaya, Juan Fernando Casanova; Corrales, Juan ...",NaN,NaN,A methodological framework proposal for managi...,FRONTIERS IN SUSTAINABLE FOOD SYSTEMS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WOS:001283555200001,0
1,J,"Ochieng, J; Kirimi, L; Ochieng, DO; Njagi, T; ...",NaN,NaN,NaN,"Ochieng, Justus; Kirimi, Lilian; Ochieng, Denn...",NaN,NaN,Managing climate risk through crop diversifica...,CLIMATIC CHANGE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WOS:000531756300001,0
2,J,"Suksa-ngiam, W; Bechor, T",NaN,NaN,NaN,"Suksa-ngiam, Watanyoo; Bechor, Tamir",NaN,NaN,The impact of applying digital process innovat...,INFORMATION DEVELOPMENT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WOS:001241998400001,0
3,J,"Danso-Abbeam, G; Ojo, TO; Baiyegunhi, LJS; Ogu...",NaN,NaN,NaN,"Danso-Abbeam, Gideon; Ojo, Temitope O.; Baiyeg...",NaN,NaN,Climate change adaptation strategies by smallh...,HELIYON,...,NaN,NaN,NaN,34151036.0,NaN,NaN,NaN,NaN,WOS:000674482300009,0
4,J,"Smith, HW; Owens, PR; Ashworth, AJ",NaN,NaN,NaN,"Smith, Harrison W.; Owens, Phillip R.; Ashwort...",NaN,NaN,Applications and Analytical Methods of Ground ...,JOURNAL OF ENVIRONMENTAL AND ENGINEERING GEOPH...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WOS:001001391900002,0


# Remove columns with any NaN values

In [54]:
data_cleaned = data.dropna(axis=1)
data_cleaned.head()

,Publication Type,Authors,Author Full Names,Article Title,Source Title,Abstract,Publication Year,DOI,DOI Link,UT (Unique WOS ID),Web of Science Record
0,J,"Olaya, JFC; Corrales, JC","Olaya, Juan Fernando Casanova; Corrales, Juan ...",A methodological framework proposal for managi...,FRONTIERS IN SUSTAINABLE FOOD SYSTEMS,Introduction Climate change and weather variab...,2024,10.3389/fsufs.2024.1363744,0,WOS:001283555200001,0
1,J,"Ochieng, J; Kirimi, L; Ochieng, DO; Njagi, T; ...","Ochieng, Justus; Kirimi, Lilian; Ochieng, Denn...",Managing climate risk through crop diversifica...,CLIMATIC CHANGE,Climatic variability and change continue to mi...,2020,10.1007/s10584-020-02727-0,0,WOS:000531756300001,0
2,J,"Suksa-ngiam, W; Bechor, T","Suksa-ngiam, Watanyoo; Bechor, Tamir",The impact of applying digital process innovat...,INFORMATION DEVELOPMENT,Thailand's agricultural sector is the lowest-p...,2024,10.1177/02666669241258920,0,WOS:001241998400001,0
3,J,"Danso-Abbeam, G; Ojo, TO; Baiyegunhi, LJS; Ogu...","Danso-Abbeam, Gideon; Ojo, Temitope O.; Baiyeg...",Climate change adaptation strategies by smallh...,HELIYON,Non-farm employment in agrarian communities in...,2021,10.1016/j.heliyon.2021.e07162,0,WOS:000674482300009,0
4,J,"Smith, HW; Owens, PR; Ashworth, AJ","Smith, Harrison W.; Owens, Phillip R.; Ashwort...",Applications and Analytical Methods of Ground ...,JOURNAL OF ENVIRONMENTAL AND ENGINEERING GEOPH...,The use of ground penetrating radar (GPR) for ...,2022,10.32389/JEEG22-001,0,WOS:001001391900002,0


# Dataset Information 

In [56]:
print(data_cleaned.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Publication Type       24 non-null     object
 1   Authors                24 non-null     object
 2   Author Full Names      24 non-null     object
 3   Article Title          24 non-null     object
 4   Source Title           24 non-null     object
 5   Abstract               24 non-null     object
 6   Publication Year       24 non-null     int64 
 7   DOI                    24 non-null     object
 8   DOI Link               24 non-null     int64 
 9   UT (Unique WOS ID)     24 non-null     object
 10  Web of Science Record  24 non-null     int64 
dtypes: int64(3), object(8)
memory usage: 2.2+ KB
None


# Building the LDA Model *Still Testing*

In [74]:
abstracts = data_cleaned['Abstract'].values

# Tokenise and clean the text
stop_words = set(stopwords.words('english'))

def preprocess(text):
    tokens = word_tokenize(text.lower())  # Convert to lowercase and tokenize
    tokens = [word for word in tokens if word.isalpha()]  # Remove punctuation and numbers
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    return tokens

# Apply preprocessing to all abstracts
processed_abstracts = [preprocess(abstract) for abstract in abstracts]

# Step 2: Prepare the Corpus
# Create a dictionary and a corpus
dictionary = corpora.Dictionary(processed_abstracts)

# Filter out extremes (optional, based on frequency)
dictionary.filter_extremes(no_below=5, no_above=0.5)

# Convert the documents into bag-of-words format
corpus = [dictionary.doc2bow(text) for text in processed_abstracts]

# Step 3: Build the LDA Model
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, random_state=42, passes=10)

# Print the topics
for idx, topic in lda_model.print_topics(-1):
    print(f"Topic {idx}: {topic}")

# Step 4: Visualize the LDA Model
# pyLDAvis to visualize the topics
pyLDAvis.enable_notebook()
lda_visualization = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(lda_visualization)

Topic 0: 0.104*"climate" + 0.097*"strategies" + 0.086*"adaptation" + 0.078*"change" + 0.038*"study" + 0.031*"used" + 0.029*"factors" + 0.021*"smallholder" + 0.021*"agriculture" + 0.021*"potential"
Topic 1: 0.096*"yield" + 0.062*"productivity" + 0.062*"crop" + 0.051*"smallholder" + 0.047*"model" + 0.032*"maize" + 0.031*"based" + 0.026*"using" + 0.025*"farmer" + 0.024*"countries"
Topic 2: 0.063*"soil" + 0.039*"water" + 0.031*"different" + 0.031*"methods" + 0.029*"new" + 0.029*"maize" + 0.028*"results" + 0.027*"study" + 0.023*"however" + 0.022*"also"
Topic 3: 0.091*"digital" + 0.089*"technologies" + 0.054*"farming" + 0.042*"used" + 0.041*"integrated" + 0.034*"use" + 0.027*"approach" + 0.027*"different" + 0.026*"lack" + 0.017*"provide"
Topic 4: 0.081*"crop" + 0.060*"agriculture" + 0.044*"technologies" + 0.042*"farming" + 0.036*"change" + 0.030*"small" + 0.030*"process" + 0.025*"management" + 0.025*"system" + 0.025*"use"
